In [1]:
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing,feature_selection
## for explainer
from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

In [2]:
from sklearn import feature_selection
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [3]:
review=pd.read_csv('Review_project_sentiment.csv')
review.head(3)

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,useful_level,language,text_clean,read_ease,polarity,subjectivity,Adj_ratio
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,moderate useful,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000,0.120000
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,moderate useful,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499,0.225888
2,7607157,wlLQ42QMHiQMJISCTUVX5A,aQA2eIdEC3_ZDHwUregu8A,wygOtij7aEb7dTdTFFnOcw,5.0,0,0,0,Took a VIP tour with DeMon of the Atlanta Merc...,2020-02-04 16:05:20,not useful,en,took vip tour demon atlanta mercedes benz stad...,-32.40,0.422500,0.696111,0.218750


In [25]:
#encoder
review1=review
review1.loc[review1['useful_level']=="useful",'useful_level']=2
review1.loc[review1['useful_level']=="moderate useful",'useful_level']=1
review1.loc[review1['useful_level']=="not useful",'useful_level']=0

#encoder1=LabelEncoder()
#review1['useful_level'] = encoder1.fit_transform(review1['useful_level'].values)
review1.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,useful_level,language,text_clean,read_ease,polarity,subjectivity,Adj_ratio
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,1,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000,0.120000
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,1,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499,0.225888
2,7607157,wlLQ42QMHiQMJISCTUVX5A,aQA2eIdEC3_ZDHwUregu8A,wygOtij7aEb7dTdTFFnOcw,5.0,0,0,0,Took a VIP tour with DeMon of the Atlanta Merc...,2020-02-04 16:05:20,0,en,took vip tour demon atlanta mercedes benz stad...,-32.40,0.422500,0.696111,0.218750
3,141509,OMw52-Eo-BYFHdUldLMc5Q,mmr-P0i__uQJHGXXeVjXxg,0Ng-wDNyA-96uaJZP8EKZg,4.0,40,0,0,When I first came here for the first time last...,2015-03-09 22:42:06,2,en,first came first time last year best friend ap...,83.96,0.168452,0.419841,0.168831
4,7962637,RWq78T9FIyauVHaHj7yl9g,CyCI71vLDa923zOMeDDA5w,WUfZXjTtBUbezJE7LnWABw,3.0,0,0,0,"I only got the take-out, so I cannot speak of ...",2013-03-22 20:47:40,0,en,got takeout cannot speak service food good wou...,51.86,0.320000,0.780000,0.200000


In [80]:
review1.to_csv('review_sentiment_encoded.csv')

从这里开始

In [4]:
review1=pd.read_csv('review_sentiment_encoded.csv')

In [5]:
## Tf-Idf (advanced variant of BoW)
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=5000, ngram_range=(1,1))

In [6]:
## split dataset
dtf_train, dtf_test = model_selection.train_test_split(review1, test_size=0.3)
## get target
y_train = dtf_train["useful_level"].values
y_test = np.array(dtf_test["useful_level"].values)

In [7]:
corpus1 = dtf_train["text_clean"]
vectorizer.fit(corpus1)
X_train = vectorizer.transform(corpus1)
dic_vocabulary = vectorizer.vocabulary_

In [8]:
X_train.shape

(38630, 5000)

In [9]:
y = dtf_train["useful_level"]
X_names = vectorizer.get_feature_names()
p_value_limit = 0.95
dtf_features = pd.DataFrame()
for cat in np.unique(y):
    chi2, p = feature_selection.chi2(X_train, y==cat)
    dtf_features = dtf_features.append(pd.DataFrame(
                   {"feature":X_names, "score":1-p, "y":cat}))
    dtf_features = dtf_features.sort_values(["y","score"], 
                    ascending=[True,False])
    dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
X_names = dtf_features["feature"].unique().tolist()

In [10]:
for cat in np.unique(y):
    print("# {}:".format(cat))
    print("  . selected features:",
         len(dtf_features[dtf_features["y"]==cat]))
    print("  . top features:", ",".join(
dtf_features[dtf_features["y"]==cat]["feature"].values[:10]))
    print(" ")

# 0:
  . selected features: 1595
  . top features: amazing,company,food,great,atmosphere,call,told,fee,doctor,review
 
# 1:
  . selected features: 575
  . top features: great,food,apartment,car,said,call,doctor,office,told,dog
 
# 2:
  . selected features: 427
  . top features: park,ride,tattoo,ron,food,great,disney,lash,contract,store
 


In [11]:
# refit vecorizor
vectorizer = feature_extraction.text.TfidfVectorizer(vocabulary=X_names)
vectorizer.fit(corpus1)
X_train = vectorizer.transform(corpus1)
dic_vocabulary = vectorizer.vocabulary_

In [12]:
corpus2 = dtf_test["text_clean"]
X_test= vectorizer.transform(corpus2)

In [13]:
classifier = naive_bayes.MultinomialNB()

In [14]:
## pipeline
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])
## train classifier
model["classifier"].fit(X_train, y_train.astype(int))
## test
X_test = dtf_test["text_clean"].values
predicted = model.predict(X_test)
#predicted = np.argmax(predicted,axis=1)
predicted_prob = model.predict_proba(X_test)

In [15]:
from sklearn.metrics import f1_score, precision_score, recall_score
f1 = f1_score( y_test, predicted, average='macro' )
p = precision_score(y_test, predicted, average='macro')
r = recall_score(y_test, predicted, average='macro')

print(f1, p, r)
# output: 0.555555555556 0.5 0.666666666667


0.49220393865072726 0.5538004007579068 0.5088770956104559


In [66]:
from sklearn.metrics import confusion_matrix

In [17]:
accuracy = metrics.accuracy_score(y_test, predicted)
auc = metrics.roc_auc_score(y_test, predicted_prob, 
                            multi_class="ovr")
print("Accuracy:",  round(accuracy,2))
print("Auc:", round(auc,2))
print("Detail:")
print(metrics.classification_report(y_test, predicted))

Accuracy: 0.57
Auc: 0.72
Detail:
              precision    recall  f1-score   support

           0       0.62      0.81      0.70      7208
           1       0.49      0.54      0.52      5247
           2       0.55      0.17      0.26      4102

    accuracy                           0.57     16557
   macro avg       0.55      0.51      0.49     16557
weighted avg       0.56      0.57      0.53     16557

